In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from os import listdir
import os
import sys
sys.path.insert(0, os.path.abspath('..')) # Note that this line is useless with a regular pip installation of PyWST.
import pywst as pw
%matplotlib notebook




In [2]:
def PS_3D(dat_,box_size,npix,v_max,v_min,delta_v,freq_min,freq_max,nbins):
    #path_cs == Path to cosmological signal fits file 
    #npix == Number of pixels n (in n x n slice)
    #box_size == Box size to the nearest Mpc
    #v_max == Highest Frequency in range (MHz)
    #v_min == Lowest Frequency in range (MHz)
    #delta_v == Frequency step in MHz
    #v_interest == frequency the power spectrum is to be taken at (nearest 0.5MHz)
    #delta_freq == Frequency slice 
    #nbins == number of bins
    import numpy as np
    from numpy import fft as f 
    from astropy.io import fits
    import os 
    import matplotlib.pyplot as plt
    from astropy.cosmology import FlatLambdaCDM,WMAP9
    import astropy.units as u 
    import numpy as np
    from numpy import fft as f 
    import math 
    from astropy.io import fits
    import matplotlib.pyplot as plt
    import sys 
    box_size = np.float(box_size)
    freq_range_ = np.arange(v_min,v_max+delta_v, delta_v) 
    f1 = freq_range_<=freq_max
    f2 = freq_min <=freq_range_
    fb = f1&f1
    b = freq_min
    delta_freq = freq_min-freq_max
    dat = dat_[fb]
    freq_range = freq_range_[fb]
    if delta_freq > 10:
        print('Having a bandwidth of 10MHz leads to more evolution in the signal!')
    H_0 = 69.32#Hubble constant 
    omega_m = WMAP9.Om0
    n_slices = (dat.shape)[0]
    zmin=1420/v_min - 1
    zmax=1420/v_max - 1
    dmin  = WMAP9.comoving_distance(zmin)
    dmax  = WMAP9.comoving_distance(zmax)
    delta_d = abs(dmax-dmin)
    k_small_perp = (2*np.pi)/box_size 
    k_large_perp = npix*k_small_perp #length of box side in k-space 
    k_small_los = (2*(np.pi)/delta_d)*u.Mpc
    k_large_los = k_small_los*n_slices 
    dat_3D = dat
    ft3d = f.fftn(dat_3D)/np.int(dat_3D.size)
    ft3d = f.fftshift(ft3d)
    PS3D = np.abs(ft3d)**2  
    if k_small_los<k_small_perp:
        k_small = k_small_los
    else:
        k_small = k_small_perp    
    if k_large_los<k_large_perp:
        k_largest = k_large_perp
    else:
        k_largest = k_large_los
    bins = np.logspace(math.log10(k_small),math.log10(k_largest ), nbins+1)
    dk_xy = (box_size/npix)/(2*np.pi)
    dk_z = (delta_d/n_slices)/(2*np.pi)
    nu_xy = np.fft.fftshift(np.fft.fftfreq(npix, dk_xy)) 
    nu_z = np.fft.fftshift(np.fft.fftfreq(n_slices, dk_z)) 
    kx, ky = np.meshgrid(nu_xy,nu_xy)
    cube_z = np.zeros((len(nu_z),kx.shape[0],kx.shape[1]))
    for o,c in zip(nu_z,range(len(nu_z))):
        cube_z[c,:,:] = o
    cube_z = np.array(cube_z)  
    ps = np.array(PS3D) 
    kmod = np.zeros((cube_z.shape[0],cube_z.shape[1],cube_z.shape[2]))
    for p in range(cube_z.shape[0]):
        kmod[p,:,:] = np.sqrt(kx**2 + ky**2 + cube_z[p]**2)
    kernel_1 = []
    kernel_2 = []
    k = np.zeros(nbins)
    for p in range(len(bins)-1):
        c1 = bins[p] <=kmod
        c2 =kmod[c1]< bins[p+1] 
        kernel_1.append(c1)
        kernel_2.append(c2)
        k[p] = bins[p+1]*100/H_0
    binned_data =np.zeros(nbins) #The binning of the FT
    ps_err=np.zeros(nbins)
    for bin_num in range(len(bins)-1):
        p1 = ps[kernel_1[bin_num]]
        bin_data = p1[kernel_2[bin_num]]
        volme = (box_size*box_size*delta_d)/u.Mpc
        prefac = (volme/(2*(np.pi)**2))   
        binned_data[bin_num] = (prefac*(np.mean(bin_data))*k[bin_num]**3)               
        PSerror =np.var(bin_data)/len(bin_data)
        ps_err[bin_num] =  PSerror          
    r = 3*(box_size/npix)
    maxkplot =   (2*np.pi)/(r)      
    tl = np.array(k) < maxkplot
    k= np.array(k)[tl]
    binned_data = np.array(binned_data)[tl]  
    ps_err = np.array(ps_err)[tl] 
    return binned_data,k,ps_err

In [3]:
def PS_3D_Coeval(dat_,box_size,npix,freq_min,freq_max,nbins):
    #path_cs == Path to cosmological signal fits file 
    #npix == Number of pixels n (in n x n slice)
    #box_size == Box size to the nearest Mpc
    #v_max == Highest Frequency in range (MHz)
    #v_min == Lowest Frequency in range (MHz)
    #delta_v == Frequency step in MHz
    #v_interest == frequency the power spectrum is to be taken at (nearest 0.5MHz)
    #delta_freq == Frequency slice 
    #nbins == number of bins
    import numpy as np
    from numpy import fft as f 
    from astropy.io import fits
    import os 
    import matplotlib.pyplot as plt
    from astropy.cosmology import FlatLambdaCDM,WMAP9
    import astropy.units as u 
    import numpy as np
    from numpy import fft as f 
    import math 
    from astropy.io import fits
    import matplotlib.pyplot as plt
    import sys 
    box_size = np.float(box_size)
    if delta_freq > 10:
        print('Having a bandwidth of 10MHz leads to more evolution in the signal!')
    n_slices = (dat.shape)[0]

    delta_d = box_size
    k_small_perp = (2*np.pi)/box_size 
    k_large_perp = npix*k_small_perp #length of box side in k-space 
    k_small_los = (2*(np.pi)/delta_d)*u.Mpc
    k_large_los = k_small_los*n_slices 
    dat_3D = dat
    ft3d = f.fftn(dat_3D)/np.int(dat_3D.size)
    ft3d = f.fftshift(ft3d)
    PS3D = np.abs(ft3d)**2  
    if k_small_los<k_small_perp:
        k_small = k_small_los
    else:
        k_small = k_small_perp    
    if k_large_los<k_large_perp:
        k_largest = k_large_perp
    else:
        k_largest = k_large_los
    bins = np.logspace(math.log10(k_small),math.log10(k_largest ), nbins+1)
    dk_xy = (box_size/npix)/(2*np.pi)
    dk_z = (delta_d/n_slices)/(2*np.pi)
    nu_xy = np.fft.fftshift(np.fft.fftfreq(npix, dk_xy)) 
    nu_z = np.fft.fftshift(np.fft.fftfreq(n_slices, dk_z)) 
    kx, ky = np.meshgrid(nu_xy,nu_xy)
    cube_z = np.zeros((len(nu_z),kx.shape[0],kx.shape[1]))
    for o,c in zip(nu_z,range(len(nu_z))):
        cube_z[c,:,:] = o
    cube_z = np.array(cube_z)  
    ps = np.array(PS3D) 
    kmod = np.zeros((cube_z.shape[0],cube_z.shape[1],cube_z.shape[2]))
    for p in range(cube_z.shape[0]):
        kmod[p,:,:] = np.sqrt(kx**2 + ky**2 + cube_z[p]**2)
    kernel_1 = []
    kernel_2 = []
    k = np.zeros(nbins)
    for p in range(len(bins)-1):
        c1 = bins[p] <=kmod
        c2 =kmod[c1]< bins[p+1] 
        kernel_1.append(c1)
        kernel_2.append(c2)
        k[p] = bins[p+1]*100/H_0
    binned_data =np.zeros(nbins) #The binning of the FT
    ps_err=np.zeros(nbins)
    for bin_num in range(len(bins)-1):
        p1 = ps[kernel_1[bin_num]]
        bin_data = p1[kernel_2[bin_num]]
        volme = (box_size*box_size*delta_d)/u.Mpc
        prefac = (volme/(2*(np.pi)**2))   
        binned_data[bin_num] = (prefac*(np.mean(bin_data))*k[bin_num]**3)               
        PSerror =np.var(bin_data)/len(bin_data)
        ps_err[bin_num] =  PSerror          
    #r = 3*(box_size/npix)
    #maxkplot =   (2*np.pi)/(r)      
    #tl = np.array(k) < maxkplot
    #k= np.array(k)[tl]
    #binned_data = np.array(binned_data)[tl]  
    #ps_err = np.array(ps_err)[tl] 
    return binned_data,k,ps_err

In [4]:
def read_lightcone(path,x_pix,y_pix,z_pix, type = np.float32):
    try :
        cube = np.fromfile(path, dtype=type)
    except FileNotFoundError :
        print(" !!!!!! file not found : "+ path)
        print("moving on...")
    
    cube =np.reshape(cube, (x_pix,y_pix,int(z_pix))).T
    return cube

In [5]:
ddir = '/Users/ianhothi/WST/Testing/pywst/examples/data/'
plt_dir = '/Users/ianhothi/Dropbox/WST_RWST/Figures/Romain_Benoit_Sim_Test/'

In [6]:
simu00004 = np.zeros((4,256,256,256))
simu00010 = np.zeros((4,256,256,256))
simu00012 = np.zeros((4,256,256,256))
simu00013 = np.zeros((4,256,256,256))
simu00014 = np.zeros((4,256,256,256))
simu00016 = np.zeros((4,256,256,256))
simu00022 = np.zeros((4,256,256,256))

In [7]:
simu00004[0,:,:,:] = np.array(read_cube(ddir+'simu00004'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00004[1,:,:,:] = np.array(read_cube(ddir+'simu00004'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00004[2,:,:,:] = np.array(read_cube(ddir+'simu00004'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00004[3,:,:,:] = np.array(read_cube(ddir+'simu00004'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

NameError: name 'read_cube' is not defined

In [8]:
simu00010[0,:,:,:] = np.array(read_cube(ddir+'simu00010'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00010[1,:,:,:] = np.array(read_cube(ddir+'simu00010'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00010[2,:,:,:] = np.array(read_cube(ddir+'simu00010'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00010[3,:,:,:] = np.array(read_cube(ddir+'simu00010'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

NameError: name 'read_cube' is not defined

In [9]:
simu00012[0,:,:,:] = np.array(read_cube(ddir+'simu00012'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00012[1,:,:,:] = np.array(read_cube(ddir+'simu00012'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00012[2,:,:,:] = np.array(read_cube(ddir+'simu00012'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00012[3,:,:,:] = np.array(read_cube(ddir+'simu00012'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

NameError: name 'read_cube' is not defined

In [10]:
simu00013[0,:,:,:] = np.array(read_cube(ddir+'simu00013'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00013[1,:,:,:] = np.array(read_cube(ddir+'simu00013'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00013[2,:,:,:] = np.array(read_cube(ddir+'simu00013'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00013[3,:,:,:] = np.array(read_cube(ddir+'simu00013'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

NameError: name 'read_cube' is not defined

In [ ]:
simu00014[0,:,:,:] = np.array(read_cube(ddir+'simu00014'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00014[1,:,:,:] = np.array(read_cube(ddir+'simu00014'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00014[2,:,:,:] = np.array(read_cube(ddir+'simu00014'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00014[3,:,:,:] = np.array(read_cube(ddir+'simu00014'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

In [ ]:
simu00016[0,:,:,:] = np.array(read_cube(ddir+'simu00016'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat'))
simu00016[1,:,:,:] = np.array(read_cube(ddir+'simu00016'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat'))
simu00016[2,:,:,:] = np.array(read_cube(ddir+'simu00016'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat'))
simu00016[3,:,:,:] = np.array(read_cube(ddir+'simu00016'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat'))

In [ ]:
simu00022[0,:,:,:] = read_cube(ddir+'simu00022'+'/'+'dtb_tp_hi_256_nocorrection_out010.dat')
simu00022[1,:,:,:] = read_cube(ddir+'simu00022'+'/'+'dtb_tp_hi_256_nocorrection_out020.dat')
simu00022[2,:,:,:] = read_cube(ddir+'simu00022'+'/'+'dtb_tp_hi_256_nocorrection_out030.dat')
simu00022[3,:,:,:] = read_cube(ddir+'simu00022'+'/'+'dtb_tp_hi_256_nocorrection_out040.dat')